## Using google maps API to evaluate circuity factor

### Load required libraries

In [ ]:
# !pip install openpyxl
# !pip install xlsxwriter

In [1]:
import pandas as pd
import googlemaps 
from datetime import datetime
from time import process_time, perf_counter, time
from math import radians, cos, sin, asin, sqrt
# import openpyxl

### Setting up the Google Maps API

In [3]:
# Load googlemaps by using the API key
# API keys are generated in the 'Credentials' page, refer to following:
# https://developers.google.com/maps/documentation/geocoding/get-api-key
gmaps = googlemaps.Client(key="Add Your Key here")

### Load both origins and destinations data available on excel file 

In [ ]:
wb_origins = pd.read_excel('distances_matrix_input.xlsx', sheet_name='origins', engine='openpyxl')

wb_destinations = pd.read_excel('distances_matrix_input.xlsx', sheet_name='destinations', engine='openpyxl')

In [ ]:
# Visualize origin points
wb_origins 

In [ ]:
# Visualize destination points
wb_destinations 

In [ ]:
print("Number of combinations Origin x Destinations: ", len(wb_destinations) * len(wb_origins))

### Define auxiliary functions

In [4]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

### Start running the evaluation

In [ ]:
distances = [["Origin",	"Lat0",	"Lon0",	"Destination",	"Lat1",	"Lon1",	"Driving Distance (m)", "Euclidean Distance (m)", "CF"]]

for index, l_origin in wb_origins.iterrows():
    for index2, l_destination in wb_destinations.iterrows():
        
        getAPI = gmaps.distance_matrix(
            str(l_origin['Lat0']) + ", " + str(l_origin['Lon0']), 
            str(l_destination['Lat1']) + ", " + str(l_destination['Lon1'])
            )
        
        d_maps = getAPI['rows'][0]['elements'][0]['distance']['value']
        
        euclidean = 1000 * haversine(lat1=l_origin['Lat0'],
                                     lon1=l_origin['Lon0'],
                                     lat2=l_destination['Lat1'],
                                     lon2=l_destination['Lon1'])

        if abs(euclidean) > 1e-6:
            circuity_factor = d_maps / euclidean 
        else:
            circuity_factor = None

        distances.append(
            [str(l_origin['Origin']), l_origin['Lat0'], l_origin['Lon0'], 
            l_destination['Destination'], l_destination['Lat1'], l_destination['Lon1'], 
            d_maps, euclidean, circuity_factor]
            )


Write Excel file whitin distance matrix combinations

In [ ]:
dist_df = pd.DataFrame(distances)
with pd.ExcelWriter('evdist_circle_v1.xlsx', engine='xlsxwriter') as writer:
    dist_df.to_excel(writer, sheet_name='distances')
    writer.save()

Segunda analise

In [21]:
wb = pd.read_excel('CF_CoordinatesComplete_v1.xlsx', engine='openpyxl')

In [22]:
wb

,T.L.,Origin,Lat0,Lon0,Destination,Lat1,Lon1,Driving Distance (m),Euclidian Distance (m),C.F.
0,91600004-91600478,91600004,-23.446472,-46.516648,91600478,-23.444269,-46.514168,NaN,NaN,NaN
1,91600004-91600481,91600004,-23.446472,-46.516648,91600481,-23.448253,-46.513729,NaN,NaN,NaN
2,91600004-91600629,91600004,-23.446472,-46.516648,91600629,-23.442377,-46.521229,NaN,NaN,NaN
3,91600004-91600639,91600004,-23.446472,-46.516648,91600639,-23.449230,-46.516488,NaN,NaN,NaN
4,91600004-91600642,91600004,-23.446472,-46.516648,91600642,-23.441844,-46.515988,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
61867,CDD-91611018,CDD,-23.435424,-46.387575,91611018,-23.507389,-46.345410,NaN,NaN,NaN
61868,CDD-91603232,CDD,-23.435424,-46.387575,91603232,-23.446692,-46.547141,NaN,NaN,NaN
61869,CDD-91609505,CDD,-23.435424,-46.387575,91609505,-23.467108,-46.514980,NaN,NaN,NaN
61870,CDD-91603777,CDD,-23.435424,-46.387575,91603777,-23.462168,-46.428795,NaN,NaN,NaN


In [23]:
distances = [["Origin",	"Lat0",	"Lon0",	"Destination",	"Lat1",	"Lon1",	"Driving Distance (m)", "Euclidean Distance (m)", "CF"]]

# i = 1

# initial_wall_time = time()
# initial_cpu_time = process_time()

# out = display('Starting', display_id=True, )

for index, line in wb.iterrows():
    #if not np.isnan(line['Lat0']) and not np.isnan(line['Lat1']):
    #if type(line['Lat0'])==float and type(line['Lat1'])==float:
        #print("aqui")
        #print(str(line['Lat0']) + ", " + str(line['Lon0']), str(line['Lat1']) + ", " + str(line['Lon1']))
    getAPI = gmaps.distance_matrix(str(line['Lat0']) + ", " + str(line['Lon0']), str(line['Lat1']) + ", " + str(line['Lon1']))
    d_maps = getAPI['rows'][0]['elements'][0]['distance']['value']
    #print(d_maps)
    euclidean = 1000* haversine(lat1=line['Lat0'],
                                lon1=line['Lon0'],
                                lat2=line['Lat1'],
                                lon2=line['Lon1'])
    #print(euclidean)
    #else:
    #    d_maps = 0    
    #    euclidean = 0
    
    # euclidean = 1000* haversine(lat1=line['Lat0'],
    #                                 lon1=line['Lon0'],
    #                                 lat2=line['Lat1'],
    #                                 lon2=line['Lon1'])

    if abs(euclidean) > 0.000001:
        circuity_factor = d_maps / euclidean 
        distances.append([str(line['Origin']), line['Lat0'], line['Lon0'], line['Destination'], line['Lat1'], line['Lon1'], d_maps, euclidean, circuity_factor]
        )
    else:
        circuity_factor = None
        distances.append([str(line['Origin']), line['Lat0'], line['Lon0'], line['Destination'], line['Lat1'], line['Lon1'], d_maps, euclidean, circuity_factor]
        )

    # Register time
    # out.update(f"Current iteration: {i:06d} | Average Time per Iteration: {(process_time() - initial_cpu_time)/i:2.6f} s")
    # i+=1

### Save the result file

In [24]:
dist_df = pd.DataFrame(distances)
with pd.ExcelWriter('CF_export_test.xlsx', engine='xlsxwriter') as writer:
    dist_df.to_excel(writer, sheet_name='distances')
    writer.save()

C:\Users\guiga\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
